In [10]:
import yt
import numpy as np
import yaml
import pandas as pd

In [11]:
# Load configuration
stream = open('config.yaml', 'r')
configs = yaml.safe_load(stream)
progenitor_directory = configs['progenitor_directory']
template_directory = configs['template_directory']
runs_directory = configs['runs_directory']

# Constants in CGS units
M_sun = 1.989E33 # Mass of the sun in grams
R_sun = 6.959E10 # Radius of the sun in centimeters
sigma_b = 5.669E-5 # Stefan-Boltzmann constant

# Easy way to convert values into the correct format for MESA model files
def format_number(number):
    return f"{number:.16e}".replace('e', 'D')

In [12]:
def shift_to_cell_center(data):
    """
    Finds the values of an array at the cell center, assuming initial values are at the edge of each cell.

    Parameters:
        data : numpy array (e.g., mass_density)

    Returns:
        shifted_array : numpy array
    """

    shifted_data = np.zeros_like(data)
    shifted_data[:-1] = data[:-1] + 0.5 * (data[1:] - data[:-1])
    shifted_data[-1] = data[-1] + 0.5 * (data[-1] - data[-2])
    return shifted_data

class StirOutput():

    # TODO: Pull some info from progenitor, such as star age and initial redshift
    def __init__(self, path):
        '''Reads data from a STIR checkpoint aat the path and stores the necessary variables.'''
        
        # Pull important data from the STIR checkpoint
        data = yt.load(path).all_data()
        self.mass_density = shift_to_cell_center(data['density'].v * 1.0)
        self.radius = shift_to_cell_center(data['r'].v * 1.0)
        self.pressure = shift_to_cell_center(data['pressure'].v * 1.0)
        self.temp = shift_to_cell_center(data['temp'].v * 1.0)
        self.radial_velocity = shift_to_cell_center(data['velx'].v * 1.0)
        self.ye = shift_to_cell_center(data['ye  '].v * 1.0) # Electron fraction
        self.volume  = shift_to_cell_center(data['cell_volume'].v * 1.0)
        self.specific_internal_energy = shift_to_cell_center(data['eint'].v * 1.0)
        self.total_specific_energy = shift_to_cell_center(data['ener'].v * 1.0)
        self.egrav = shift_to_cell_center(data['gravitational_potential'].v * 1.0)
        self.grav_potential = shift_to_cell_center(data['gpot'].v * 1.0) # What was the difference between this and the above?

        # Calculate some additional values
        self.total_energy = (self.total_specific_energy - 1.275452534480232E+018 + self.grav_potential) * self.mass_density  * self.volume
        self.cell_count = len(self.mass_density)
    
        # Determines the PNS mass cut by finding the first unbound cell and summing the mass below that radius
        bound_index = np.min(np.where(self.total_energy > 0.0))
        self.pns_masscut = np.sum(self.mass_density[:bound_index]  * self.volume[:bound_index])

        # Data that will be written in table form in the done_with_edep.mod file
        self.mesa_table_data = pd.DataFrame({
            
            # IMPLEMENTED
            'lnd': np.log(self.mass_density),
            'lnT': np.log(self.temp),
            'lnR': np.log(self.radius),

            # TODO: Need to calculate
            'L': np.zeros(self.cell_count), # TODO: Should I calculate this using the total energy and some dynamical time?
            'dq': np.zeros(self.cell_count), # TODO: Fraction of xmstar=(mstar-mcenter)
            'u': np.zeros(self.cell_count), # TODO: Cell center riemann velocity
            'mlt_vc': np.zeros(self.cell_count), # TODO: MLT convection velocity

            # TODO: Mass Fractions for atomic particles and elements
            'neut': np.zeros(self.cell_count),
            'h1': np.zeros(self.cell_count),
            'prot': np.zeros(self.cell_count),
            'he3': np.zeros(self.cell_count),
            'he4': np.zeros(self.cell_count),
            'c12': np.zeros(self.cell_count),
            'n14': np.zeros(self.cell_count),
            'o16': np.zeros(self.cell_count),
            'ne20': np.zeros(self.cell_count),
            'mg24': np.zeros(self.cell_count),
            'si28': np.zeros(self.cell_count),
            's32': np.zeros(self.cell_count),
            'ar36': np.zeros(self.cell_count),
            'ca40': np.zeros(self.cell_count),
            'ti44': np.zeros(self.cell_count),
            'cr48': np.zeros(self.cell_count),
            'cr60': np.zeros(self.cell_count),
            'fe52': np.zeros(self.cell_count),
            'fe54': np.zeros(self.cell_count),
            'fe56': np.zeros(self.cell_count),
            'co56': np.zeros(self.cell_count),
            'ni56': np.zeros(self.cell_count)

        })  

    def setup_mesa_input(self, model_index):
        '''Takes the STIR output data and writes the necessary information to the MESA input files.''' 

        # IMPLEMENTED
        n_shells = ' ' * (25 - int(np.floor(np.log10(self.cell_count)))) + str(self.cell_count)
        total_mass = np.sum(self.mass_density * self.volume) / M_sun # In units of solar mass
        total_energy = np.sum(self.total_energy) # TODO: Is it okay that the total energy is negative?
        model_number = ' ' * (25 - int(np.floor(np.log10(model_index)))) + str(model_index)

        # TODO: UNIMPLEMENTED, Pull from progenitor
        star_age = 6.3376175628057906E-09
        initial_z = 2.0000000000000000E-02

        # TODO: UNIMPLEMENTED, Need to calculate
        R_center = 3.9992663820663236E+07 # Look to flash_to_snec code
        Teff = 7.2257665281116360E+03 # Use the stefan-boltzmann law after luminosity is calculated
        xmstar = 2.0363416138072876E+34 # How do I calcualte this?
        core_mass = 3.0136524023699760E+33 # Same method as R_center

        # TODO: Do I need to bother?
        # Cumulative energy error
        # Powers that are all 0 in the template header anyway
        # Number of species (may affect number of columns mesa reads?)
        # Average core density?

        file_header = f"""! note: initial lines of file can contain comments
!
           548 -- model for mesa/star, cell center Riemann velocities (u), mlt convection velocity (mlt_vc). cgs units. lnd=ln(density), lnT=ln(temperature), lnR=ln(radius), L=luminosity, dq=fraction of xmstar=(mstar-mcenter) in cell; remaining cols are mass fractions.

                  version_number   'r24.08.1'
                          M/Msun      {format_number(total_mass)}
                    model_number      {model_number}
                        star_age      {format_number(star_age)}
                       initial_z      {format_number(initial_z)}
                        n_shells      {n_shells}
                        net_name   'approx21_cr60_plus_co56.net'
                         species                              22
                          xmstar      {format_number(xmstar)}  ! above core (g).  core mass: Msun, grams:      {format_number(core_mass / M_sun)}    {format_number(core_mass)}
                        R_center      {format_number(R_center)}  ! radius of core (cm).  R/Rsun, avg core density (g/cm^3):      {format_number(R_center / R_sun)}    1.1247695543683205D+10
                            Teff      {format_number(Teff)}
                  power_nuc_burn      0.0000000000000000D+00
                    power_h_burn      0.0000000000000000D+00
                   power_he_burn      0.0000000000000000D+00
                    power_z_burn      0.0000000000000000D+00
                     power_photo      0.0000000000000000D+00
                    total_energy      {format_number(total_energy)}
         cumulative_energy_error      1.3995525796409981D+42
   cumulative_error/total_energy      1.7494407245495471D-09  log_rel_run_E_err     -8.7571007679208037D+00
                     num_retries                               0

                lnd                        lnT                        lnR                          L                         dq                          u                     mlt_vc                   neut                       h1                         prot                       he3                        he4                        c12                        n14                        o16                        ne20                       mg24                       si28                       s32                        ar36                       ca40                       ti44                       cr48                       cr60                       fe52                       fe54                       fe56                       co56                       ni56     
"""

        # Add one line for each cell, consisting of all it's properties
        new_lines = []
        for line_index in range(self.cell_count):

            # Writes the cell/line index 
            spaces = 4 - int(np.floor(np.log10(line_index + 1)))
            new_line = ' ' * spaces + str(line_index + 1)

            # Writes each of the properties
            for column_name in self.mesa_table_data.columns:
                spaces = 5 if self.mesa_table_data.at[line_index, column_name] > 0 else 4
                new_line += spaces * ' ' + format_number(self.mesa_table_data.at[line_index, column_name])

            new_lines.append(new_line)

        # Footer containing info about the previous model
        file_footer = f"""
        
        previous model

               previous n_shells      {n_shells}
           previous mass (grams)      2.3377068540442852D+34 
              timestep (seconds)      4.2684831294390047D-04 
               dt_next (seconds)      5.1221797553268049D-04 """

        # Write all of the above to the done_with_edep.mod file
        with open(f'model_{model_index}/done_with_edep.mod', 'w') as file:
            file.writelines(file_header + '\n'.join(new_lines) + file_footer)

In [13]:
stir_output = StirOutput("z9p6_sep4_hyb_a1.0_hdf5_chk_0511")

yt : [INFO     ] 2025-01-20 17:12:15,238 Particle file found: z9p6_sep4_hyb_a1.0_hdf5_chk_0511
yt : [WARNING  ] 2025-01-20 17:12:15,250 Extending phi dimension to 2PI + left edge.
yt : [WARNING  ] 2025-01-20 17:12:15,250 Extending theta dimension to PI + left edge.
yt : [INFO     ] 2025-01-20 17:12:15,299 Parameters: current_time              = 5.090000287824025
yt : [INFO     ] 2025-01-20 17:12:15,300 Parameters: domain_dimensions         = [128   1   1]
yt : [INFO     ] 2025-01-20 17:12:15,300 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-01-20 17:12:15,301 Parameters: domain_right_edge         = [2.40000000e+10 3.14159265e+00 6.28318531e+00]
yt : [INFO     ] 2025-01-20 17:12:15,302 Parameters: cosmological_simulation   = 0


In [14]:
stir_output.setup_mesa_input(1)